In [17]:
# 1️⃣ 라이브러리
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time

# 2️⃣ URL 설정
URL = "https://www.hyundai.com/kr/ko/gw/customer-support/v1/customer-support/faq/list"
SOURCE_URL = "https://www.hyundai.com/kr/ko/customer-support/faq"
HEADERS = {"Content-Type": "application/json;charset=UTF-8"}

# 3️⃣ HTML 클린 함수 (div 단위 줄바꿈만)
def clean_html(html: str) -> str:
    """HTML 텍스트를 클린 처리: <div> 단위 줄바꿈, <br> 줄바꿈"""
    soup = BeautifulSoup(html, "html.parser")
    
    # <br> -> 줄바꿈
    for br in soup.find_all('br'):
        br.replace_with("\n")
    
    # <div> 단위 줄바꿈
    for div in soup.find_all('div'):
        div.insert_before("\n")
    
    # 텍스트 추출
    return soup.get_text("\n", strip=True)

# 4️⃣ FAQ 크롤링
rows = []

for cat in range(1, 10):  # 현대 카테고리 9개
    category_code = f"{cat:02}"
    page = 1

    while True:
        payload = {
            "siteTypeCode": "H",
            "faqCategoryCode": category_code,
            "faqCode": "",
            "faqSeq": "",
            "searchKeyword": "",
            "pageNo": str(page),
            "pageSize": "50",
            "externalYn": ""
        }

        r = requests.post(URL, json=payload, headers=HEADERS)
        data = r.json()

        faq_list = data["data"]["list"]
        if not faq_list:
            break

        for item in faq_list:
            # 질문/답변 HTML 가져오기
            q_html = item.get("faqQuestion", "")
            a_html = item.get("faqAnswer", "")

            # 클린 처리
            q_text = clean_html(q_html)
            a_text = clean_html(a_html)

            # 행 추가
            rows.append({
                "company": "hyundai",
                "category": item.get("faqCategoryName"),
                "category_code": item.get("faqCategoryCode"),
                "question": q_text,       # 클린 처리된 질문
                "answer_html": a_html,    # 원본 HTML
                "answer_text": a_text,    # 클린 처리된 텍스트
                "image_urls": [],         # 필요 시 이미지 추가 가능
                "source": SOURCE_URL
            })

        page += 1
        time.sleep(0.2)  # 서버 과부하 방지

# 5️⃣ DataFrame 생성
hyundai_df = pd.DataFrame(rows)
hyundai_df["company"] = "hyundai"

In [19]:
hyundai_df.to_pickle("FAQ_Hyundai.pkl")